In [4]:
#!pip install chromadb
#!pip install sentence-transformers

import chromadb

# Create a client
client = chromadb.Client()

# Create or get a collection to store your vectors
collection = client.create_collection(name="my_collection")
texts = ["Hello world", "How are you?", "Chroma DB is cool!"]
embeddings = [
    [0.1, 0.2, 0.3],  # example embedding vector for "Hello world"
    [0.4, 0.1, 0.9],
    [0.9, 0.7, 0.2]
]
collection = client.create_collection(name="my_collection", get_or_create=True)

# Add data with unique IDs, embeddings, and metadata (optional)
collection.add(
    ids=["1", "2", "3"],
    documents=texts,
    embeddings=embeddings
)
query_embedding = [0.1, 0.2, 0.25]

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=2  # how many closest results to return
)

print(results)
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(texts)


InternalError: Collection [my_collection] already exists

In [2]:
!pip install tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 932.9 kB/s eta 0:00:02
   ------------------------ --------------- 1.0/1.7 MB 1.2 MB/s eta 0:00:01
   ------------------------ --------------- 1.0/1.7 MB 1.2 MB/s eta 0:00:01
   ------------------------------ --------- 1.3/1.7 MB 1.1 MB/s eta 0:00:01
   ------------------------------ --------- 1.3/1.7 MB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 976.4 kB/s eta 0:00:00


In [4]:
!pip install faiss-cpu

   ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
    --------------------------------------- 0.3/15.0 MB ? eta -:--:--
    --------------------------------------- 0.3/15.0 MB ? eta -:--:--
    --------------------------------------- 0.3/15.0 MB ? eta -:--:--
    --------------------------------------- 0.3/15.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/15.0 MB 322.8 kB/s eta 0:00:45
   -- ------------------------------------- 0.8/15.0 MB 541.1 kB/s eta 0:00:27
   ---- ----------------------------------- 1.6/15.0 MB 1.0 MB/s eta 0:00:13
   ---- ----------------------------------- 1.8/15.0 MB 1.2 MB/s eta 0:00:12
   ------ --------------------------------- 2.6/15.0 MB 1.4 MB/s eta 0:00:09
   --------- ------------------------------ 3.4/15.0 MB 1.7 MB/s eta 0:00:07
   ----------- ---------------------------- 4.5/15.0 MB 2.0 MB/s eta 0:00:06
   --------------- ------------------

In [5]:
!pip install PyMuPDF

   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
   - -------------------------------------- 0.8/18.7 MB 4.2 MB/s eta 0:00:05
   -- ------------------------------------- 1.3/18.7 MB 3.0 MB/s eta 0:00:06
   -- ------------------------------------- 1.3/18.7 MB 3.0 MB/s eta 0:00:06
   --- ------------------------------------ 1.6/18.7 MB 2.3 MB/s eta 0:00:08
   --- ------------------------------------ 1.8/18.7 MB 1.9 MB/s eta 0:00:09
   ---- ----------------------------------- 2.1/18.7 MB 1.8 MB/s eta 0:00:10
   ----- ---------------------------------- 2.4/18.7 MB 1.6 MB/s eta 0:00:11
   ----- ---------------------------------- 2.6/18.7 MB 1.5 MB/s eta 0:00:11
   ------ --------------------------------- 2.9/18.7 MB 1.5 MB/s eta 0:00:11
   ------ --------------------------------- 3.1/18.7 MB 1.6 MB/s eta 0:00:11
   ------- -------------------------------- 3.4/18.7 MB 1.5 MB/s eta 0:00:11
   ------- -------------------------------- 3.7/18.7 MB 1.5 MB/s eta 0:00:11
   ---

In [7]:
!pip install pdfplumber

   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   ---------------------------------------- 5.6/5.6 MB 49.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
   ---------------------------------------- 3.0/3.0 MB 57.4 MB/s eta 0:00:00


In [8]:
from sentence_transformers import SentenceTransformer
import faiss
import fitz  # PyMuPDF
import pdfplumber
import numpy as np
import requests
import os

def download_pdf_from_url(url, filename="temp.pdf"):
    response = requests.get(url)
    if response.status_code == 200 and response.headers["Content-Type"] == "application/pdf":
        with open(filename, 'wb') as f:
            f.write(response.content)
        return filename
    else:
        raise Exception("Failed to download PDF. Check the URL.")

def extract_pdf_data(pdf_path):
    doc = fitz.open(pdf_path)
    data = {"texts": [], "images": [], "tables": []}
    
    # Extract text & tables
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            data["texts"].append(page.extract_text())
            data["tables"].extend(page.extract_tables())

    return data

model = SentenceTransformer('all-MiniLM-L6-v2')

def create_embeddings(text_chunks):
    return model.encode(text_chunks, convert_to_numpy=True)

def build_faiss_index(embeddings):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)
    return index

def search_index(index, query, documents, top_k=3):
    query_vector = model.encode([query])
    D, I = index.search(query_vector, top_k)
    return [documents[i] for i in I[0]]


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\student\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\student\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# Provide your PDF URL here
pdf_url = "https://world-heart-federation.org/wp-content/uploads/World-Heart-Report-2023.pdf"

pdf_file = download_pdf_from_url(pdf_url)
pdf_data = extract_pdf_data(pdf_file)

# Combine all texts & tables into a single document list
documents = [t for t in pdf_data["texts"] if t] + [str(table) for table in pdf_data["tables"]]
embeddings = create_embeddings(documents)
index = build_faiss_index(embeddings)

# Now ask questions
query = "What is the summary of this document?"
results = search_index(index, query, documents)

for r in results:
    print("🔹", r)

# Clean up downloaded file
os.remove(pdf_file)


🔹 [['', 'Measure: Age-standardised total physical activity (MET-min/week). The Metabolic\nEquivalent of Task (MET) metric measures how much energy an activity consumes\ncompared to being at rest. The amount of MET minutes per week expresses how much\nenergy one has expended while performing various activities throughout the whole week.\nData: Global Burden of Disease, 2019']]
🔹 [['', 'Measure: Population weighted average of particulate matter PM2.5, µg/m3\nData: Global Burden of Diseases, 2019']]
🔹 [['', 'Measure: Mean age-standardized alcohol consumption expressed in grams/day\nData: Global Burden of Diseases, 2019']]


In [11]:
!pip install torch torchvision

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from t
device = torc
latent_dim = 100
batch_size = 128
lr = 0.0002
epochs = 10
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]) 
])
dataset = datasets.MNIST('.', download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 28*28),
            nn.Tanh()  # Output pixels in [-1, 1]
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), 1, 28, 28)
        return img

# Discriminator Model
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()  
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

generator = Generator().to(device)
discriminator = Discriminator().to(device)
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=lr)
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr)

for epoch in range(epochs):
    for i, (imgs, _) in enumerate(dataloader):

        batch_size_curr = imgs.size(0)
        real_imgs = imgs.to(device)

        # Ground truths
        valid = torch.ones(batch_size_curr, 1, device=device)
        fake = torch.zeros(batch_size_curr, 1, device=device)

        optimizer_G.zero_grad()

        z = torch.randn(batch_size_curr, latent_dim, device=device)
        generated_imgs = generator(z)
        validity = discriminator(generated_imgs)
        g_loss = criterion(validity, valid) 

        g_loss.backward()
        optimizer_G.step()

        optimizer_D.zero_grad()

        real_loss = criterion(discriminator(real_imgs), valid)
        fake_loss = criterion(discriminator(generated_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        if i % 400 == 0:
            print(f"Epoch [{epoch+1}/{epochs}] Batch {i}/{len(dataloader)} \
                  Loss D: {d_loss.item():.4f}, loss G: {g_loss.item():.4f}")

print("Training complete!")


100%|█████████████████████████████████████████████████████████████████████████████| 9.91M/9.91M [00:06<00:00, 1.52MB/s]
100%|██████████████████████████████████████████████████████████████████████████████| 28.9k/28.9k [00:00<00:00, 116kB/s]
100%|██████████████████████████████████████████████████████████████████████████████| 1.65M/1.65M [00:09<00:00, 183kB/s]
100%|█████████████████████████████████████████████████████████████████████████████████████| 4.54k/4.54k [00:00<?, ?B/s]


Epoch [1/10] Batch 0/469                   Loss D: 0.6687, loss G: 0.6524
Epoch [1/10] Batch 400/469                   Loss D: 0.0128, loss G: 5.4833
Epoch [2/10] Batch 0/469                   Loss D: 0.0182, loss G: 3.9398
Epoch [2/10] Batch 400/469                   Loss D: 0.0131, loss G: 6.8855
Epoch [3/10] Batch 0/469                   Loss D: 0.0143, loss G: 7.6642
Epoch [3/10] Batch 400/469                   Loss D: 0.2897, loss G: 2.1877
Epoch [4/10] Batch 0/469                   Loss D: 0.2487, loss G: 2.3750
Epoch [4/10] Batch 400/469                   Loss D: 1.1186, loss G: 0.9525
Epoch [5/10] Batch 0/469                   Loss D: 0.3849, loss G: 2.3081
Epoch [5/10] Batch 400/469                   Loss D: 0.2539, loss G: 3.4629
Epoch [6/10] Batch 0/469                   Loss D: 0.1962, loss G: 2.3114
Epoch [6/10] Batch 400/469                   Loss D: 0.4530, loss G: 1.3092
Epoch [7/10] Batch 0/469                   Loss D: 0.2040, loss G: 1.7119
Epoch [7/10] Batch 400/469

In [15]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import requests
import torch

# Load model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Load image
img_url = "https://example.com/image.jpg"
image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

# Process image for image encoder only
inputs = processor(images=image, return_tensors="pt")

# Extract image embeddings using model's vision_encoder directly
with torch.no_grad():
    image_embeds = model.vision_encoder(pixel_values=inputs['pixel_values']).pooler_output

# Normalize embeddings
image_embeds = torch.nn.functional.normalize(image_embeds, p=2, dim=1)

print(image_embeds.shape)  # Should be [1, embedding_dim]




from sentence_transformers import SentenceTransformer
import torch

# Load sentence embedding model for captions
caption_embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Get image embedding from BLIP model
image_embeds = model.get_image_features(**inputs)

# Normalize embeddings
image_embeds = torch.nn.functional.normalize(image_embeds, p=2, dim=1)
caption_embeds = caption_embedder.encode([caption], convert_to_tensor=True)
caption_embeds = torch.nn.functional.normalize(caption_embeds, p=2, dim=1)


import faiss
import numpy as np

# Convert to numpy arrays
image_vec = image_embeds.cpu().detach().numpy()
caption_vec = caption_embeds.cpu().detach().numpy()

# Initialize FAISS index for 512-dim (depends on your embedding size)
dim = image_vec.shape[1]
index = faiss.IndexFlatIP(dim)  # Using inner product (cosine similarity)

# Add image embeddings to the index
index.add(image_vec)

id_to_caption = {0: caption}

query = "A dog playing in the park"
query_vec = caption_embedder.encode([query], convert_to_numpy=True)
query_vec = query_vec / np.linalg.norm(query_vec)

# Search in FAISS index
k = 5  # top 5 results
D, I = index.search(query_vec, k)

for i in I[0]:
    print(f"Caption: {id_to_caption[i]}")


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x0000017A9D7C9580>